# Objetivo del código
Este código realiza dos acciones fundamentalmente:   

1.   Filtrar qué notas (.txt) tienen al menos una anotación válida
2.   Separar las anotaciones de cada task en ficheros independientes

El input de este script son las notas del conjunto MOD anotadas con BRAT


## Parte I
Cuando hacemos la anotación con BRAT, hay notas (.txt) que no tienen ninguna entidad. Sin embargo, BRAT por defecto crea archivos .ann de esas notas en blanco (pesan 0 Kb).

Código para quedarnos únicamente con los archivos .txt que tengan un archivo .ann válido (que tenga al menos una entidad). Estos archivos se obtienen tras borrar los .ann que pesan 0 kb (No hay ninguna entidad)

**Es decir, del total de notas (.txt) que conforman el CORPUS MOD queremos aislar sólo aquellas que tienen una entidad anotada. La forma de saber qué notas tienen una entidad es: a) quedarnos con los archivos .ann más grandes de 0 kb, b) coger los nombres de esos archivos (sin la extensión) c) buscar los archivos .txt que tengan los mismos nombres y moverlos a una carpeta aislada**

In [ ]:
# Cargamos las librerías necesarias
import numpy as np
import pandas as pd
import re
import os, glob
from sklearn.datasets import load_files
import shutil
# Dado que hacemos muchas operaciones de lectura y escritura debemos añadir delays para dar tiempo a que se completen
import time

# Habilitamos display interactivo y enlace con nuestra cuenta de google
from google.colab import data_table
from google.colab import drive

# Enlazamos nuestro notebook en Colab con nuestro almacenamiento en Google Drive 
drive.mount('/content/drive')

Mounted at /content/drive


En la carpeta `/content/drive/MyDrive/Colab Notebooks/Corpus/MOD` ponemos tanto las notas .txt como los archivos .ann, sacadas directamente de BRAT

In [ ]:
# Define relative path to folder containing the text files
files_folder = "/content/drive/MyDrive/Colab Notebooks/Corpus/MOD"
notas = []

# Create a dataframe list by using a list comprehension
#notas = [pd.read_csv(file, delimiter='\t') for file in glob.glob(os.path.join(files_folder ,"*.txt"))]
#notas = pd.DataFrame(notas, columns = ['notas'])

In [ ]:
# Comprobamos que la ruta esté bien
files_folder

'/content/drive/MyDrive/Colab Notebooks/Corpus/MOD'

Creamos listas que contengan los nombres de los ficheros .ann y .txt

In [ ]:
# https://stackoverflow.com/questions/3207219/how-do-i-list-all-files-of-a-directory
import glob

# Creamos una lista con la ruta de todos los archivos .ann contenidos en la carpeta
archivos_ann_rutaCompleta = glob.glob("/content/drive/MyDrive/Colab Notebooks/Corpus/MOD/*.ann")

# Con esta instrucción nos quedamos únicamente con el nombre del archivo sin la extensión
archivos = [os.path.splitext(os.path.basename(x))[0] for x in archivos_ann_rutaCompleta]

# Al nombre del archivo le añadimos la extensión .txt 
archivos_txt = [s + ".txt" for s in archivos]
archivos_ann = [s + ".ann" for s in archivos]

#len(archivos)

# Vemos un ejemplo del contenido de archivos
print(archivos[:10])
print(archivos_txt[:10])
print(archivos_ann[:10])
print(archivos_ann_rutaCompleta[:10])

['32119083_ES', '32617223_ES', 'caso_clinico_atencion_primaria12', 'caso_clinico_dermatologia20', '32318729_ES', '32329981_ES', 'caso_clinico_atencion_primaria155', 'caso_clinico_atencion_primaria76', 'caso_clinico_atencion_primaria126', 'caso_clinico_dermatologia419']
['32119083_ES.txt', '32617223_ES.txt', 'caso_clinico_atencion_primaria12.txt', 'caso_clinico_dermatologia20.txt', '32318729_ES.txt', '32329981_ES.txt', 'caso_clinico_atencion_primaria155.txt', 'caso_clinico_atencion_primaria76.txt', 'caso_clinico_atencion_primaria126.txt', 'caso_clinico_dermatologia419.txt']
['32119083_ES.ann', '32617223_ES.ann', 'caso_clinico_atencion_primaria12.ann', 'caso_clinico_dermatologia20.ann', '32318729_ES.ann', '32329981_ES.ann', 'caso_clinico_atencion_primaria155.ann', 'caso_clinico_atencion_primaria76.ann', 'caso_clinico_atencion_primaria126.ann', 'caso_clinico_dermatologia419.ann']
['/content/drive/MyDrive/Colab Notebooks/Corpus/MOD/32119083_ES.ann', '/content/drive/MyDrive/Colab Notebooks/

In [ ]:
# Vemos que cogemos realmente las anotaciones
archivos_ann_rutaCompleta[:10]

['/content/drive/MyDrive/Colab Notebooks/Corpus/MOD/32119083_ES.ann',
 '/content/drive/MyDrive/Colab Notebooks/Corpus/MOD/32617223_ES.ann',
 '/content/drive/MyDrive/Colab Notebooks/Corpus/MOD/caso_clinico_atencion_primaria12.ann',
 '/content/drive/MyDrive/Colab Notebooks/Corpus/MOD/caso_clinico_dermatologia20.ann',
 '/content/drive/MyDrive/Colab Notebooks/Corpus/MOD/32318729_ES.ann',
 '/content/drive/MyDrive/Colab Notebooks/Corpus/MOD/32329981_ES.ann',
 '/content/drive/MyDrive/Colab Notebooks/Corpus/MOD/caso_clinico_atencion_primaria155.ann',
 '/content/drive/MyDrive/Colab Notebooks/Corpus/MOD/caso_clinico_atencion_primaria76.ann',
 '/content/drive/MyDrive/Colab Notebooks/Corpus/MOD/caso_clinico_atencion_primaria126.ann',
 '/content/drive/MyDrive/Colab Notebooks/Corpus/MOD/caso_clinico_dermatologia419.ann']

In [ ]:
# Vemos el número de casos clínicos que conforman el corpus
len(archivos)

265

Creamos rutas donde vamos a guardar los archivos

In [ ]:
# Establecemos la ruta de la carpeta de origen
carpeta_origen = "/content/drive/MyDrive/Colab Notebooks/Corpus/MOD"

# Creamos el directorio
from pathlib import Path
Path("/content/drive/MyDrive/Colab Notebooks/Corpus/MOD/Task1").mkdir(parents=True, exist_ok=True)
Path("/content/drive/MyDrive/Colab Notebooks/Corpus/MOD/Task2").mkdir(parents=True, exist_ok=True)

# Establecemos la ruta de la carpeta de destino
carpeta_destino_task1 = "/content/drive/MyDrive/Colab Notebooks/Corpus/MOD/Task1"
carpeta_destino_task2 = "/content/drive/MyDrive/Colab Notebooks/Corpus/MOD/Task2"

Copiamos los archivos .txt que contengan al menos una anotación a las rutas creadas anteriormente, además también copiamos las anotaciones.

In [ ]:
# Copiamos todos los archivos .txt que tienen una nota .ann a una carpeta nueva
# Al poner archivos_ann como extremo me aseguro de que sólo voy a copiar archivos .txt que tengan al menos una entidad
# Porque todos los archivos .ann tienen un .txt correspondiente
for i in range(0, len(archivos_ann)):
  shutil.copyfile(carpeta_origen + "/" + archivos_txt[i], carpeta_destino_task1 + "/" + archivos_txt[i])
  shutil.copyfile(carpeta_origen + "/" + archivos_txt[i], carpeta_destino_task2 + "/" + archivos_txt[i])
  shutil.copyfile(carpeta_origen + "/" + archivos_ann[i], carpeta_destino_task1 + "/" + archivos_ann[i])
  shutil.copyfile(carpeta_origen + "/" + archivos_ann[i], carpeta_destino_task2 + "/" + archivos_ann[i])
  print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264


Una vez hemos movido los archivos a las carpetas correspondientes los eliminamos de la carpeta original (MOD)

In [ ]:
# Dejamos que pasen cinco minutos desde la última operación hasta su sincronización
time.sleep(5*60)

In [ ]:
# Eliminamos los archivos .txt
filelist = glob.glob(os.path.join("/content/drive/MyDrive/Colab Notebooks/Corpus/MOD/", "*.txt"))
for f in filelist:
    os.remove(f)

# Eliminamos los archivos .ann
filelist = glob.glob(os.path.join("/content/drive/MyDrive/Colab Notebooks/Corpus/MOD/", "*.ann"))
for f in filelist:
    os.remove(f)

In [ ]:
# Dejamos que pase un minuto desde la última operación hasta su sincronización
time.sleep(60)

## Parte II

Las anotaciones en BRAT se han hecho a la vez para las dos tasks. Por lo tanto, en un mismo archivo .ann aparecen anotaciones de PROFESIÓN (task 1) y anotaciones de a QUIEN pertenece la profesión (task 2). Tenemos que crear un script que nos permita filtrar las anotaciones de la task 1 y almacenarlas en una carpeta y filtrar las de la task 2 y almacenarlas en otra carpeta.

Creamos dos carpetas en procesado: Task1 y Task2. En ambas carpetas guardamos/movemos las notas .txt y las notas .ann (es decir se duplican y se almacenan en cada carpeta)

### TASK 1

In [ ]:
# Fijamos la ruta donde se encuentran los archivos de los cuales queremos obtener las anotaciones
# Puede ser .tsv si estamos en entrenamiento o development (desconozco porque se usaron dos formatos distintos)

#TASK 1
path = r'/content/drive/MyDrive/Colab Notebooks/Corpus/MOD/Task1/*.ann'

# Creamos una lista con todos los ficheros a transformar
files = glob.glob(path)
len(files)

265

En las carpetas Task1 y Task2 creamos la carpeta modificado donde vamos metiendo los archivo .ann procesados (es decir, en la tarea 1 conservando las anotaciones de ocupaciones y en la tarea 2 conservando las anotaciones de a quien hace referencia)

In [ ]:
# Creamos un bucle para ir procesando cada fichero
for i in range(0,len(files)):
    # Mostramos archivo que se está procesando
    print(files[i])
    # Leemos los archivos. En el caso de que haya alguna línea con errores de lectura la omitimos.
    # En este paso obviamos la especificación de un separador (puede dar problemas), por lo que ponemos uno que nunca se dará
    # para que se lea la línea entera
    archivo = pd.read_csv(files[i], header = None, names = ["col"], on_bad_lines='skip', sep = "~")
    
    # Nos quedamos únicamente con las anotaciones relacionadas con ocupaciones
    # TASK 1
    archivo = archivo[archivo.col.str.contains("PROFESION|SITUACION_LABORAL|ACTIVIDAD")]
    ruta = "/content/drive/MyDrive/Colab Notebooks/Corpus/MOD/Task1/" + files[i].replace("/content/drive/MyDrive/Colab Notebooks/Corpus/MOD/Task1/", "")
    archivo.to_csv(ruta,index=False,header=False)
    
    # NO EJECUTAR
    #Si quisiéramos guardar el archivo en otra carpeta
    #ruta = "/content/drive/MyDrive/Colab Notebooks/Corpus/MOD/Task1/modificado/" + files[i].replace("/content/drive/MyDrive/Colab Notebooks/Corpus/MOD/Task1/", "")
    #archivo.to_csv(ruta,index=False,header=False)

/content/drive/MyDrive/Colab Notebooks/Corpus/MOD/Task1/32119083_ES.ann
/content/drive/MyDrive/Colab Notebooks/Corpus/MOD/Task1/32617223_ES.ann
/content/drive/MyDrive/Colab Notebooks/Corpus/MOD/Task1/caso_clinico_atencion_primaria12.ann
/content/drive/MyDrive/Colab Notebooks/Corpus/MOD/Task1/caso_clinico_dermatologia20.ann
/content/drive/MyDrive/Colab Notebooks/Corpus/MOD/Task1/32318729_ES.ann
/content/drive/MyDrive/Colab Notebooks/Corpus/MOD/Task1/32329981_ES.ann
/content/drive/MyDrive/Colab Notebooks/Corpus/MOD/Task1/caso_clinico_atencion_primaria155.ann
/content/drive/MyDrive/Colab Notebooks/Corpus/MOD/Task1/caso_clinico_atencion_primaria76.ann
/content/drive/MyDrive/Colab Notebooks/Corpus/MOD/Task1/caso_clinico_atencion_primaria126.ann
/content/drive/MyDrive/Colab Notebooks/Corpus/MOD/Task1/caso_clinico_dermatologia419.ann
/content/drive/MyDrive/Colab Notebooks/Corpus/MOD/Task1/caso_clinico_dermatologia451.ann
/content/drive/MyDrive/Colab Notebooks/Corpus/MOD/Task1/caso_clinico_der

### TASK 2

In [ ]:
# Fijamos la ruta donde se encuentran los archivos de los cuales queremos obtener las anotaciones
# Puede ser .tsv si estamos en entrenamiento o development (desconozco porque se usaron dos formatos distintos)

# TASK 2
path = r'/content/drive/MyDrive/Colab Notebooks/Corpus/MOD/Task2/*.ann'

# Creamos una lista con todos los ficheros a transformar
files = glob.glob(path)
len(files)

265

In [ ]:
# Para ejecutar este código comentar y descomentar las líneas en función de la TASK que se quiera realizar

# Creamos un bucle para ir procesando cada fichero
for i in range(0,len(files)):
    # Mostramos archivo que se está procesando
    print(files[i])
    # Leemos los archivos. En el caso de que haya alguna línea con errores de lectura la omitimos.
    # En este paso obviamos la especificación de un separador (puede dar problemas), por lo que ponemos uno que nunca se dará
    # para que se lea la línea entera
    archivo = pd.read_csv(files[i], header = None, names = ["col"], on_bad_lines='skip', sep = "~")
    
    #TASK 2
    archivo = archivo[archivo.col.str.contains("PACIENTE|FAMILIAR|SANITARIO|OTROS")]
    ruta = "/content/drive/MyDrive/Colab Notebooks/Corpus/MOD/Task2/" + files[i].replace("/content/drive/MyDrive/Colab Notebooks/Corpus/MOD/Task2/", "")
    archivo.to_csv(ruta,index=False,header=False)
    
    # NO EJECUTAR
    #Si quisiéramos guardar el archivo en otra carpeta
    #ruta = "/content/drive/MyDrive/Colab Notebooks/Corpus/MOD/Task2/modificado/" + files[i].replace("/content/drive/MyDrive/Colab Notebooks/Corpus/MOD/Task2/", "")
    #archivo.to_csv(ruta,index=False,header=False)


/content/drive/MyDrive/Colab Notebooks/Corpus/MOD/Task2/32119083_ES.ann
/content/drive/MyDrive/Colab Notebooks/Corpus/MOD/Task2/32617223_ES.ann
/content/drive/MyDrive/Colab Notebooks/Corpus/MOD/Task2/caso_clinico_atencion_primaria12.ann
/content/drive/MyDrive/Colab Notebooks/Corpus/MOD/Task2/caso_clinico_dermatologia20.ann
/content/drive/MyDrive/Colab Notebooks/Corpus/MOD/Task2/32318729_ES.ann
/content/drive/MyDrive/Colab Notebooks/Corpus/MOD/Task2/32329981_ES.ann
/content/drive/MyDrive/Colab Notebooks/Corpus/MOD/Task2/caso_clinico_atencion_primaria155.ann
/content/drive/MyDrive/Colab Notebooks/Corpus/MOD/Task2/caso_clinico_atencion_primaria76.ann
/content/drive/MyDrive/Colab Notebooks/Corpus/MOD/Task2/caso_clinico_atencion_primaria126.ann
/content/drive/MyDrive/Colab Notebooks/Corpus/MOD/Task2/caso_clinico_dermatologia419.ann
/content/drive/MyDrive/Colab Notebooks/Corpus/MOD/Task2/caso_clinico_dermatologia451.ann
/content/drive/MyDrive/Colab Notebooks/Corpus/MOD/Task2/caso_clinico_der

In [ ]:
# Ponemos este último time sleep para que el script no de por finalizado hasta que no se hayan sincronizado todos los archivos
time.sleep(3*60)